**Imports**

In [3]:
import os
import torch
import numpy as np
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
import gymnasium
import ale_py
from utils import make_env, process_state
from IPython.display import Video
from dqn_agent import DQNAgent
from dqn_cnn_model import DQN_CNN_Model
from IPython.display import clear_output
import matplotlib.pyplot as plt
from double_dqn_agent import DoubleDQNAgent


In [4]:
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic=True # https://discuss.pytorch.org/t/what-is-the-differenc-between-cudnn-deterministic-and-cudnn-benchmark/38054
torch.backends.cudnn.benchmark=True # https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936/4
np.random.seed(SEED)
random.seed(SEED)

In [5]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda"  
elif torch.backends.mps.is_available():
    DEVICE = "mps" 

In [6]:
GRAY_SCALE = True 
SCREEN_SIZE = 84 
NUM_STACKED_FRAMES = 4 
SKIP_FRAMES = 4 
ENV_NAME = "ALE/Breakout-v5" 

## Entrenamiento

In [7]:
#Hiperparámetros de entrenamiento del agente DQN
TOTAL_STEPS = 10_000_000
EPISODES = 10_000
STEPS_PER_EPISODE = 20_000

EPSILON_INI = 1
EPSILON_MIN = 0.05
EPSILON_ANNEAL_STEPS = 1_000_000

EPISODE_BLOCK = 100

BATCH_SIZE = 32
BUFFER_SIZE = 50_000

GAMMA = 0.995
LEARNING_RATE = 1e-5

In [8]:
env = make_env(ENV_NAME,
                video_folder='./videos/dqn_training',
                name_prefix="breakout",
                record_every=500,
                grayscale=GRAY_SCALE,
                screen_size=SCREEN_SIZE,
                stack_frames=NUM_STACKED_FRAMES,
                skip_frames=SKIP_FRAMES
                )

net = DQN_CNN_Model(env.observation_space.shape, env.action_space.n).to(DEVICE)

dqn_agent = DQNAgent(env, net, process_state, BUFFER_SIZE, BATCH_SIZE, LEARNING_RATE, GAMMA, 
                     epsilon_i=EPSILON_INI, epsilon_f=EPSILON_MIN, 
                     epsilon_anneal_steps=EPSILON_ANNEAL_STEPS, 
                     episode_block=EPISODE_BLOCK, device=DEVICE)



rewards_dqn = dqn_agent.train(EPISODES, STEPS_PER_EPISODE, TOTAL_STEPS)

env.close()

c:\Users\joaco\anaconda3\envs\obl_taller_ia\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\Users\joaco\Documents\ort\entregas_taller_de_ia\obligatorio\videos\dqn_training folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Training:   0%|          | 3/10000 [00:20<18:43:29,  6.74s/episode, reward=1.67, epsilon=0.999, steps=558]


KeyboardInterrupt: 

In [9]:
env = make_env(ENV_NAME,
                video_folder='./videos/ddqn_training',
                name_prefix="breakout",
                record_every=500,
                grayscale=GRAY_SCALE,
                screen_size=SCREEN_SIZE,
                stack_frames=NUM_STACKED_FRAMES,
                skip_frames=SKIP_FRAMES
                )


modelo_a = DQN_CNN_Model(env.observation_space.shape, env.action_space.n).to(DEVICE)
modelo_b = DQN_CNN_Model(env.observation_space.shape, env.action_space.n).to(DEVICE)

ddqn_agent = DoubleDQNAgent(env, modelo_a, modelo_b, process_state, BUFFER_SIZE, BATCH_SIZE, LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI, epsilon_f=EPSILON_MIN, epsilon_anneal_steps=EPSILON_ANNEAL_STEPS, episode_block = EPISODE_BLOCK, device=DEVICE)

ddqn_rewards = ddqn_agent.train(EPISODES, STEPS_PER_EPISODE, TOTAL_STEPS)

ddqn_agent.save_checkpoint("DDQN.pth")

c:\Users\joaco\anaconda3\envs\obl_taller_ia\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\Users\joaco\Documents\ort\entregas_taller_de_ia\obligatorio\videos\ddqn_training folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Training:   0%|          | 0/10000 [00:00<?, ?episode/s]c:\Users\joaco\anaconda3\envs\obl_taller_ia\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training:   0%|          | 3/10000 [00:13<12:30:36,  4.51s/episode, reward=0.333, epsilon=1, steps=386]


KeyboardInterrupt: 